In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(5)

In [2]:
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        for i in range(dim):
            self.Ix[i,i] = 1
    
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        self.fc5 = nn.Linear(m, m)
        self.fc6 = nn.Linear(m, m)
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y)**deg
        y = self.fc2(y)
        y = F.relu(y)**deg
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y)**deg
        y = self.fc4(y)
        y = F.relu(y)**deg
        y = y+s
        s=y
        y = self.fc5(y)
        y = F.relu(y)**deg
        y = self.fc6(y)
        y = F.relu(y)**deg
        y = y+s
        output = self.outlayer(y)
        return output
deg = 2

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
def generate_uniform_points_in_sphere(d,R,N):
    points = torch.randn(N,d)
    scales = (R*torch.rand(N))**(1/d)
    notes = points/(torch.sqrt(torch.sum(points**2,dim=1))/scales).reshape(N,1)*torch.ones(N ,d)
    return notes

In [5]:
def sol_exact(x):
    norm_2 = torch.sum(x**2,dim=1).reshape([x.size()[0],1])
    N_u = torch.cos(norm_2-1)
    return N_u

In [6]:
def right(x):
    norm_2 = torch.sum(x**2,dim=1).reshape([x.size()[0],1])
    N_u = 2*dim_set*torch.sin(1-norm_2)
    N_u = N_u - 4*norm_2*torch.cos(1-norm_2)
    return N_u

In [7]:
Batch_size = 10000
dim_set = 2
def DGM():
    x = generate_uniform_points_in_sphere(dim_set,1,Batch_size).cuda()
    x.requires_grad = True
    norm = (torch.sum(x**2,dim=1).reshape([x.size()[0],1]))**(1/2)
    u = model_u(x)
    v= torch.ones(u.shape).cuda()
    ux = (torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0])
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = ux[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss = torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1])+ u - sol_exact(x) -right(x))**2)/Batch_size
    return loss

In [8]:
model_u1 = ResNet(dim_set,10,1)
device=torch.device("cuda:5" )
model_u1.to(device)

ResNet(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=10, bias=True)
  (outlayer): Linear(in_features=10, out_features=1, bias=True)
)

In [9]:
def model_u(x):
    x.requires_grad = True
    N_x = model_u1(x)
    v= torch.ones(N_x.shape).cuda()
    norm_2 = torch.sum(x**2,dim=1).reshape([x.size()[0],1])
    L_N = norm_2-1
    N_v = x
    nabla_N_x = torch.autograd.grad(N_x,x,grad_outputs=v,create_graph=True)[0]
    G = -torch.sum(nabla_N_x*N_v,dim=1).reshape([x.size()[0],1])
    return L_N*G/2 + N_x

In [10]:
def loss_error():
    notes_error = generate_uniform_points_in_sphere(dim_set,1,Batch_size).cuda()
    notes_error.requires_grad= True
    exact = sol_exact(notes_error)
    u_L2 = torch.sqrt(torch.sum(exact**2)/10000)
    predict= model_u(notes_error)
    value = torch.sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    return value

In [11]:
traintime = 100000
error_save=np.zeros([traintime])
optimizer = optim.Adam([
                {'params': model_u1.parameters()}
            ])
i = 0
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses = DGM()
    error = loss_error()
    losses.backward()
    optimizer.step() 
    error_save[i]=float(error)
    i = i +1
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error)
        print("loss1 =",losses.detach())
        np.save("error_save_Neumann_s_2D_w10.npy", error_save)
np.save("error_save_Neumann_s_2D_w10.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = tensor(0.7838, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(4.9450, device='cuda:5')
i=  501
error = tensor(0.0638, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0128, device='cuda:5')
i=  1001
error = tensor(0.0845, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0061, device='cuda:5')
i=  1501
error = tensor(0.0898, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0048, device='cuda:5')
i=  2001
error = tensor(0.0744, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0058, device='cuda:5')
i=  2501
error = tensor(0.0534, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0055, device='cuda:5')
i=  3001
error = tensor(0.0381, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0038, device='cuda:5')
i=  3501
error = tensor(0.0299, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0028, device='cuda:5')
i=  4001
error = tensor(0.0227, device='cuda:5', grad_fn=<DivBackward0>)
loss1 = tensor(0.0017, devi